# Gaussian Bayesian Classifier

### Mastery Checkpoint 1  
CSC 466, Winter 2022  
Samay Nathani

In [45]:
from pathlib import Path
home = str(Path.home())

In [46]:
%load_ext autoreload
%autoreload 2

import Lab2_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import our datasets

In [47]:
import pandas as pd
import numpy as np
import math
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)

Do some preprocessing like Lab 2

In [48]:
features = ['Pclass','Survived','Sex','Age']
titanic_df = titanic_df.loc[:,features]
display(titanic_df.head())
titanic_df.loc[:,'Pclass']=titanic_df['Pclass'].fillna(titanic_df['Pclass'].mode()).astype(int)
titanic_df.loc[:,'Age']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df.loc[:,'Age']=(titanic_df['Age']/10).astype(str).str[0].astype(int)*10
titranic_df = titanic_df.dropna()
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
#probs = Lab2_helper.class_conditional(X,y)
#priors = Lab2_helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
#post_probs = Lab2_helper.posteriors(probs,priors,x)

,Pclass,Survived,Sex,Age
0,3,0,male,22.0
1,1,1,female,38.0
2,3,1,female,26.0
3,1,1,female,35.0
4,3,0,male,35.0


Defining the Gaussian Bayesian Classifier

In [90]:
class GaussianBayes:
    
    def compute_priors(self, y):
        value_counts = y.value_counts().sort_index()
        total_values = len(y)
        indexes = [y.name+"="+str(v) for v in value_counts.index.tolist()]
        priors = dict(zip(indexes, value_counts / total_values))
        return priors
    
    def compute_normal_distribution(self, mean, stddev, x):
        exp = -1*(x-mean)**2 / 2*(stddev**2)
        base = math.e / (stddev) ** math.sqrt(2 * math.pi)
        return base ** exp
    
    
    # Since we are assuming a normal distribution, this is where we use the mean, std, and variance to calculate the class conditional probability. 
    def specific_class_conditional(self,x,xv,y,yv):
        # todo: replace binning with normal distribution calculations
        likelihoods = {}
        priors = {}
        
        for ux in x.unique():
            y_vals = y.loc[x==ux]
            prob = self.compute_normal_distribution(y_vals.mean(), y_vals.std(), yv)
            likelihoods[ux] = prob
            priors[ux] = len(y_vals) / y

        denom = 0
        
        for k in priors.keys():
            denom += (priors[k]*likelihoods[k])

        print(likelihoods, priors)
        classcond = likelihoods[xv]*priors[xv] / denom
        return classcond


    def class_conditional(self,X,y):
        probs = {}
        
        for eachy in y.unique():
            for col in X.columns:
                for eachx in X[col].unique():
                    probs[col + "=" + str(eachx) + "|" + y.name + "=" + str(eachy)] = specific_class_conditional(X[col], eachx, y, eachy)
        return probs

    def posteriors(self,probs,priors,x):
        post_probs = {}
        denom = 0
        for k in priors.keys():
            numerator = 1
            postkey = ""
            
            for idx in x.index:
                postkey += idx + "=" + str(x[idx]) + ","
                probkey = idx + "=" + str(x[idx]) + "|" + k
                
                if probkey not in probs:
                    numerator *= 0
                else:
                    numerator *= probs[probkey]
            
            numerator*=priors[k]
            post_probs[k + "|" + postkey[:-1]] = numerator
            denom+=numerator
            
        for k, v in post_probs.items():
            if denom != 0:
                post_probs[k] = v / denom
            else:
                post_probs[k] = 1 / len(list(priors.keys()))
        
        return post_probs


In [91]:
gb = GaussianBayes()

In [92]:
prob = gb.specific_class_conditional(titanic_df['Sex'],'female',titanic_df['Survived'],0)

{'male': 0.9908703373000626, 'female': 0.8502683426821895} {'male': 0        inf
1      577.0
2      577.0
3      577.0
4        inf
       ...  
886      inf
887    577.0
888      inf
889    577.0
890      inf
Name: Survived, Length: 891, dtype: float64, 'female': 0        inf
1      314.0
2      314.0
3      314.0
4        inf
       ...  
886      inf
887    314.0
888      inf
889    314.0
890      inf
Name: Survived, Length: 891, dtype: float64}
